In [1]:
import os
import zipfile

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [2]:
def read_data_ml100k():
    data_dir = "ml-100k"
    names = ['user_id', 'item_id', 'rating', 'timestamp']
    data = pd.read_csv(os.path.join(data_dir, 'u.data'), sep='\t',
                       names=names, engine='python')
    num_users = data.user_id.unique().shape[0]
    num_items = data.item_id.unique().shape[0]
    return data, num_users, num_items

In [3]:
def split_data_ml100k(data, num_users, num_items,
                      split_mode='random', test_ratio=0.1):
    if split_mode == 'seq-aware':
        train_items, test_items, train_list = {}, {}, []
        for line in data.itertuples():
            u, i, rating, time = line[1], line[2], line[3], line[4]
            train_items.setdefault(u, []).append((u, i, rating, time))
            if u not in test_items or test_items[u][-1] < time:
                test_items[u] = (i, rating, time)
        for u in range(1, num_users+1):
            train_list.extend(sorted(train_items[u], key=lambda k: k[3]))
        test_data = [(key, *value) for key, value in test_items.items()]
        train_data = [item for item in train_list if item not in test_data]
        train_data = pd.DataFrame(train_data)
        test_data = pd.DataFrame(test_data)
    else:
        mask = [True if x == 1 else False for x in np.random.uniform(
            0, 1, (len(data))) < 1 - test_ratio]
        neg_mask = [not x for x in mask]
        train_data, test_data = data[mask], data[neg_mask]
    return train_data, test_data

In [4]:
def load_data_ml100k(data, num_users, num_items, feedback='explicit'):
    users, items, scores = [], [], []
    inter = np.zeros((num_items, num_users)) if feedback == 'explicit' else {}
    for line in data.itertuples():
        user_index, item_index = int(line[1] - 1), int(line[2] - 1)
        score = int(line[3]) if feedback == 'explicit' else 1
        users.append(user_index)
        items.append(item_index)
        scores.append(score)
        if feedback == 'implicit':
            inter.setdefault(user_index, []).append(item_index)
        else:
            inter[item_index, user_index] = score
    return users, items, scores, inter

In [5]:
def split_and_load_ml100k(split_mode='seq-aware', feedback='explicit',
                          test_ratio=0.1, batch_size=256):
    data, num_users, num_items = read_data_ml100k()
    train_data, test_data = split_data_ml100k(
        data, num_users, num_items, split_mode, test_ratio)
    _, _, _, train_inter = load_data_ml100k(
        train_data, num_users, num_items, feedback)
    _, _, _, test_inter = load_data_ml100k(
        test_data, num_users, num_items, feedback)
    train_iter = DataLoader(train_inter, shuffle=True, batch_size=batch_size)
    test_iter = DataLoader(test_inter, batch_size=batch_size)
    return num_users, num_items, train_iter, test_iter

In [6]:
class AutoRec(nn.Module):
    def __init__(self, num_factors, num_users, dropout=0.05):
        super().__init__()
        self.V = nn.Linear(num_users, num_factors)
        self.mu = nn.Parameter(torch.zeros(num_factors))
        self.W = nn.Linear(num_factors, num_users)
        self.b = nn.Parameter(torch.zeros(num_users))
        
        self.g_function = nn.Sigmoid()
        self.f_function = nn.Identity()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, rating, type='train'):
        latent = self.dropout(self.g_function(self.V(rating) + self.mu.unsqueeze(0).expand_as(self.V(rating))))
        outputs = self.f_function(self.W(latent) + self.b.unsqueeze(0).expand_as(self.W(latent)))
        
        if type == 'train':  # Mask the gradient during training
            return outputs * torch.sign(rating)
        else:
            return outputs
        
        # if self.training:
        #     return outputs * torch.sign(rating)
        # else:
        #     return outputs

In [7]:
# class AutoRec(nn.Module):
#     def __init__(self, num_hidden, num_users, dropout=0.05):
#         super(AutoRec, self).__init__()
#         self.encoder = nn.Sequential(
#             nn.Linear(num_users, num_hidden, bias = True),
#             nn.Sigmoid(),
#             # nn.ReLU(True),
#             nn.Dropout(dropout)
#         )
#         self.decoder = nn.Sequential(
#             nn.Linear(num_hidden, num_users, bias=True),
#             # nn.ReLU(True),
#         )
#         self.type = type

#     def forward(self, input, type='train'):
#         x = self.encoder(input)
#         pred = self.decoder(x)
#         if type == 'train':  # Mask the gradient during training
#             return pred * torch.sign(input)
#         else:
#             return pred

In [17]:
# class AutoRec(nn.Module):
#     def __init__(self, num_hidden, num_users, dropout=0.05):
#         super(AutoRec, self).__init__()
#         self.encoder = nn.Linear(num_users, num_hidden, bias=True)
#         self.sigmoid = nn.Sigmoid()
#         self.dropout = nn.Dropout(dropout)
#         self.decoder = nn.Linear(num_hidden, num_users, bias=True)

#     def forward(self, input):
#         hidden = self.dropout(self.sigmoid(self.encoder(input)))
#         pred = self.decoder(hidden)
#         if self.training:  # Mask the gradient during training
#             return pred * torch.sign(input)
#         else:
#             return pred

In [18]:
class RMSELoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, truth, prediction):
        criterion = nn.MSELoss()
        eps = 1e-6
        loss = torch.sqrt(criterion(truth, prediction) + eps)
        return loss

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_factors = 500
lr = 0.002
batch_size = 256
wd = 1e-5

num_epochs = 25

In [20]:
num_users, num_items, train_iter, test_iter = split_and_load_ml100k(batch_size=batch_size)

In [21]:
model = AutoRec(num_factors, num_users, 0.05).to(device)
loss_fn = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

In [23]:
for epoch in range(num_epochs):
    train_loss = []
    model.train()
    
    for i, values in enumerate(train_iter):
        optimizer.zero_grad()
        
        rating = values.float().to(device)
        
        preds = model(rating)
        loss = loss_fn(rating, preds)
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.detach().cpu().item())
        
    test_loss = []
    model.eval()
    with torch.no_grad():
        for i, values in enumerate(test_iter):
            
            rating = values.float().to(device)
            
            preds = model(rating)
            loss = loss_fn(rating, preds)
            test_loss.append(loss.detach().cpu().item())


    print(f"{epoch} Epochs")
    print(f"train_loss: {np.mean(train_loss):.3f}")
    print(f"test_loss: {np.mean(test_loss):.3f}")
    print("\n")

0 Epochs
train_loss: 0.326
test_loss: 2.350


1 Epochs
train_loss: 0.276
test_loss: 2.479


2 Epochs
train_loss: 0.256
test_loss: 2.623


3 Epochs
train_loss: 0.246
test_loss: 2.618


4 Epochs
train_loss: 0.234
test_loss: 2.738


5 Epochs
train_loss: 0.232
test_loss: 2.803


6 Epochs
train_loss: 0.227
test_loss: 2.793


7 Epochs
train_loss: 0.223
test_loss: 2.828


8 Epochs
train_loss: 0.218
test_loss: 2.845


9 Epochs
train_loss: 0.215
test_loss: 2.866


10 Epochs
train_loss: 0.212
test_loss: 2.880


11 Epochs
train_loss: 0.207
test_loss: 2.897


12 Epochs
train_loss: 0.204
test_loss: 2.904


13 Epochs
train_loss: 0.201
test_loss: 2.904


14 Epochs
train_loss: 0.196
test_loss: 2.932


15 Epochs
train_loss: 0.193
test_loss: 2.932


16 Epochs
train_loss: 0.191
test_loss: 2.943


17 Epochs
train_loss: 0.185
test_loss: 2.946


18 Epochs
train_loss: 0.183
test_loss: 2.945


19 Epochs
train_loss: 0.179
test_loss: 2.956


20 Epochs
train_loss: 0.175
test_loss: 2.967


21 Epochs
train_loss: 0